In [1]:
!pip install SigProfilerMatrixGenerator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 66.8 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.2.0 which is incompatible.


In [2]:
from SigProfilerMatrixGenerator import install as genInstall

In [3]:
genInstall.install('GRCh37', rsync=False, bash=True)


Tool       | Installed 
-----------------------
curl       | True      
wget       | True      
rsync      | True      
Downloading: 100.00% [780.58 MB of 780.58 MB] at 38.97 MB/s
Download complete.
All reference files have been created.
To proceed with matrix_generation, please provide the path to your vcf files and an appropriate output path.
Installation complete.


In [4]:
from SigProfilerMatrixGenerator.scripts import SigProfilerMatrixGeneratorFunc as matGen

In [10]:
import pandas as pd
BRCA_FR_SUPERFILTRATO_WG=pd.read_csv('/content/BRCA_FR_AF_superFILTRO_sample_inputmatrix.txt',sep='\t')

In [11]:
BRCA_FR_SUPERFILTRATO_WG

,Project,Sample,ID,Genome,mut_type,chrom,pos_start,pos_end,ref,alt,Type
0,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
1,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
2,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
3,BRCA-FR,SA596039,SA596039,GRCh37,SNP,10,64401763,64401763,G,T,SOMATIC
4,BRCA-FR,SA596039,SA596039,GRCh37,SNP,1,202468579,202468579,C,A,SOMATIC
...,...,...,...,...,...,...,...,...,...,...,...
418581,BRCA-FR,SA595120,SA595120,GRCh37,SNP,8,15395130,15395130,G,C,SOMATIC
418582,BRCA-FR,SA595120,SA595120,GRCh37,SNP,8,15395130,15395130,G,C,SOMATIC
418583,BRCA-FR,SA595120,SA595120,GRCh37,SNP,8,15395130,15395130,G,C,SOMATIC
418584,BRCA-FR,SA595120,SA595120,GRCh37,SNP,6,42376519,42376519,C,T,SOMATIC


In [12]:
BRCA_FR_SUPERFILTRATO_WG['chrom']='chr'+ BRCA_FR_SUPERFILTRATO_WG['chrom'].astype(str)

In [13]:
BRCA_FR_SUPERFILTRATO_WG['pos_start']=BRCA_FR_SUPERFILTRATO_WG['pos_start'].astype(int)
BRCA_FR_SUPERFILTRATO_WG['pos_end']=BRCA_FR_SUPERFILTRATO_WG['pos_end'].astype(int)

In [14]:
!pip install pysam
!pip install tqdm
!pip install biopython

In [8]:
import pysam
from tqdm import tqdm

def extract_sequences_from_fasta(fasta_file, tsv_file, output_file):
    # Apri il file TSV per leggere le posizioni
  with open(tsv_file, 'r') as tsv:
        # Salta l'intestazione
      next(tsv)

      with open(output_file, 'w') as output:
        # Apri il file FASTA
        with pysam.FastaFile(fasta_file) as fasta:
            for line in tqdm(tsv, desc='Elaborazione'):
                parts = line.strip().split('\t')
                chrom = parts[0]
                start = int(parts[1])
                end = int(parts[2])

                # Estrai la sequenza in base alle posizioni
                sequence = fasta.fetch(chrom, start, end)

                # Scrivi la sequenza in un file di output
                output.write(f'>{chrom}:{start}-{end}\n')
                output.write(sequence + '\n')


In [9]:
from Bio import SeqIO
from collections import Counter

# Definisci una funzione per generare tutte le triplette possibili
def generate_triplets():
    bases = "ACGT"
    triplets = [base1 + base2 + base3 for base1 in bases for base2 in bases for base3 in bases]
    return triplets

In [ ]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
import random
DF=pd.DataFrame()
TRIPLETS={}
for z in range(1000):
    ESONI=pd.read_csv('/content/drive/MyDrive/esoni_noduplicati.tsv',sep='\t')
    ESONI['chrom']=ESONI['chrom'].astype(str)
    ESONI['exonStarts']=ESONI['exonStarts'].astype(int)
    ESONI['exonEnds']=ESONI['exonEnds'].astype(int)
    n=random.randint(7510,30038)
    ESONI=ESONI.sample(n)
    ESONI.to_csv("esoni_target.tsv",sep='\t', index=False)
    #triplette
    fasta_file = '/content/drive/MyDrive/hg19.fa'
    tsv_file = '/content/esoni_target.tsv'
    output_file = '/content/drive/MyDrive/esoni_genoma_triplette_targettato_ANN.fa'

    extract_sequences_from_fasta(fasta_file, tsv_file, output_file)

    triplet_counter = Counter(generate_triplets())
    # Esegui il conteggio delle triplette nel file FASTA
    for record in SeqIO.parse(output_file, "fasta"):
      sequence = str(record.seq).upper()  # Converte la sequenza in maiuscolo
      for i in range(0, len(sequence) - 2, 3):
        triplet = sequence[i:i + 3]
        if triplet in triplet_counter:
          triplet_counter[triplet] += 1

    for key in triplet_counter:
       triplet_counter[key] -= 1

    def compare_blocks(block_BRCA, block_ESONI):
      merged_block = block_BRCA.merge(block_ESONI, on='chrom', how='inner')
      filtered_block = merged_block[
           (merged_block['pos_start'] >= merged_block['exonStarts']) &
           (merged_block['pos_end'] <= merged_block['exonEnds'])]
      return filtered_block[['Project', 'Sample', 'ID', 'Genome', 'mut_type', 'chrom', 'pos_start','pos_end', 'ref', 'alt', 'Type']]

    block_size = 1000

    num_blocks_BRCA = int(np.ceil(len(BRCA_FR_SUPERFILTRATO_WG) / block_size))
    num_blocks_ESONI = int(np.ceil(len(ESONI) / block_size))

    block_indices = []
    for i in range(num_blocks_BRCA):
      start_idx_BRCA = i * block_size
      end_idx_BRCA = min((i + 1) * block_size, len(BRCA_FR_SUPERFILTRATO_WG))

      for j in range(num_blocks_ESONI):
        start_idx_ESONI = j * block_size
        end_idx_ESONI = min((j + 1) * block_size, len(ESONI))

        block_indices.append([start_idx_BRCA, end_idx_BRCA, start_idx_ESONI, end_idx_ESONI])

    def compare_blocks_wrapper(indices):
        block_BRCA = BRCA_FR_SUPERFILTRATO_WG.iloc[indices[0]:indices[1]]
        block_ESONI = ESONI.iloc[indices[2]:indices[3]]
        return compare_blocks(block_BRCA, block_ESONI)

    with Pool() as pool:
      results = pool.map(compare_blocks_wrapper, block_indices)

    final_result = pd.concat(results, ignore_index=True)
    for i in range(final_result.shape[0]):
      final_result.loc[i,'chrom']= final_result.loc[i,'chrom'][3:]
    final_result.to_csv('/content/input/brca_fr_target_superfiltrato.txt', sep='\t', index=False)
    try:
      matrices = matGen.SigProfilerMatrixGeneratorFunc("test", "GRCh37", "/content",plot=False, exome=False, bed_file=None, chrom_based=False, tsb_stat=False, seqInfo=False, cushion=100)
    except ValueError as e:
      pass
    df = pd.read_table("/content/output/SBS/test.SBS96.all")
    df = df.T
    df.columns=df.iloc[0,:]
    df=df.iloc[1:73,]
    DF=DF.append(df)
    TRIPLETS[z]=triplet_counter

Elaborazione: 24676it [00:12, 1996.25it/s]


Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 3.12 seconds.
Matrices generated for 72 samples with 5651 errors. Total of 1460 SNVs, 1 DINUCs, and 0 INDELs were successfully analyzed.


Elaborazione: 11634it [00:03, 3463.98it/s]


Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 3.06 seconds.
Matrices generated for 70 samples with 2667 errors. Total of 705 SNVs, 5 DINUCs, and 0 INDELs were successfully analyzed.


Elaborazione: 12434it [00:03, 3853.03it/s]


...

Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 3.13 seconds.
Matrices generated for 72 samples with 3544 errors. Total of 990 SNVs, 2 DINUCs, and 0 INDELs were successfully analyzed.


Elaborazione: 16585it [00:03, 4281.49it/s]


Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 3.87 seconds.
Matrices generated for 71 samples with 3343 errors. Total of 916 SNVs, 4 DINUCs, and 0 INDELs were successfully analyzed.


Elaborazione: 28524it [00:05, 4980.79it/s]


KeyboardInterrupt: 

In [ ]:
DF

MutationType,A[C>A]A,A[C>A]C,A[C>A]G,A[C>A]T,A[C>G]A,A[C>G]C,A[C>G]G,A[C>G]T,A[C>T]A,A[C>T]C,...,T[T>A]G,T[T>A]T,T[T>C]A,T[T>C]C,T[T>C]G,T[T>C]T,T[T>G]A,T[T>G]C,T[T>G]G,T[T>G]T
SA594315,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SA594666,0,0,0,0,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
SA594675,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
SA594687,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SA594695,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SA596039,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SA596051,0,0,0,0,0,1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
SA596068,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SA596093,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
len(TRIPLETS)

93

In [ ]:
DF.to_csv('DF.csv')

In [ ]:
# Convertire il dizionario in una stringa JSON
import json
json_string = json.dumps(TRIPLETS, indent=2)  # indent è opzionale e rende il file più leggibile

# Salvare la stringa JSON in un file di testo
with open('TRIPLETS.txt', 'w') as file:
    file.write(json_string)

In [59]:
import pandas as pd
DF=pd.read_csv("/content/DF.csv")

In [60]:
DF

,Unnamed: 0.1,Unnamed: 0,A[C>A]A,A[C>A]C,A[C>A]G,A[C>A]T,A[C>G]A,A[C>G]C,A[C>G]G,A[C>G]T,...,T[T>A]G,T[T>A]T,T[T>C]A,T[T>C]C,T[T>C]G,T[T>C]T,T[T>G]A,T[T>G]C,T[T>G]G,T[T>G]T
0,0,SA594315,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,SA594666,0,0,0,0,1,0,0,3,...,0,0,0,0,0,0,0,0,0,0
2,2,SA594675,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,3,SA594687,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,SA594695,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70991,70991,SA596039,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
70992,70992,SA596051,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
70993,70993,SA596068,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70994,70994,SA596093,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [53]:
DF['Unnamed: 0'] = DF['Unnamed: 0'].fillna(DF['Unnamed: 0.1'])

In [54]:
DF=DF.drop(['Unnamed: 0.1'],axis=1)

In [55]:
DF['Unnamed: 0'].unique()

array(['SA594315', 'SA594666', 'SA594675', 'SA594687', 'SA594695',
       'SA594712', 'SA594737', 'SA594762', 'SA594772', 'SA594784',
       'SA594804', 'SA594818', 'SA594835', 'SA594850', 'SA594865',
       'SA594870', 'SA594881', 'SA594892', 'SA594901', 'SA594910',
       'SA594927', 'SA594933', 'SA594944', 'SA594950', 'SA594956',
       'SA594964', 'SA594970', 'SA594978', 'SA594981', 'SA594991',
       'SA595002', 'SA595011', 'SA595019', 'SA595028', 'SA595038',
       'SA595044', 'SA595059', 'SA595065', 'SA595068', 'SA595078',
       'SA595083', 'SA595093', 'SA595105', 'SA595113', 'SA595120',
       'SA595126', 'SA595140', 'SA595145', 'SA595149', 'SA595157',
       'SA595162', 'SA595167', 'SA595170', 'SA595175', 'SA595182',
       'SA595186', 'SA595189', 'SA595192', 'SA595197', 'SA595203',
       'SA595208', 'SA595213', 'SA595220', 'SA595225', 'SA595995',
       'SA596016', 'SA596027', 'SA596039', 'SA596051', 'SA596068',
       'SA596093', 'SA596106'], dtype=object)

In [56]:
DF

,Unnamed: 0,A[C>A]A,A[C>A]C,A[C>A]G,A[C>A]T,A[C>G]A,A[C>G]C,A[C>G]G,A[C>G]T,A[C>T]A,...,T[T>A]G,T[T>A]T,T[T>C]A,T[T>C]C,T[T>C]G,T[T>C]T,T[T>G]A,T[T>G]C,T[T>G]G,T[T>G]T
0,SA594315,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SA594666,0,0,0,0,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
2,SA594675,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,SA594687,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SA594695,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70991,SA596039,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
70992,SA596051,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
70993,SA596068,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70994,SA596093,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [34]:
import json
with open("/content/TRIPLETS.txt", 'r') as file:
    contenuto_file = file.read()

# Parsifica il contenuto in un dizionario
TRIPLETS = json.loads(contenuto_file)

In [35]:
len(TRIPLETS)

1000

In [22]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
import random

for z in range(500,600):
    ESONI=pd.read_csv('/content/drive/MyDrive/esoni_noduplicati.tsv',sep='\t')
    ESONI['chrom']=ESONI['chrom'].astype(str)
    ESONI['exonStarts']=ESONI['exonStarts'].astype(int)
    ESONI['exonEnds']=ESONI['exonEnds'].astype(int)
    n=random.randint(7510,30038)
    ESONI=ESONI.sample(n)
    ESONI.to_csv("esoni_target.tsv",sep='\t', index=False)
    #triplette
    fasta_file = '/content/drive/MyDrive/hg19.fa'
    tsv_file = '/content/esoni_target.tsv'
    output_file = '/content/drive/MyDrive/esoni_genoma_triplette_targettato_ANN.fa'

    extract_sequences_from_fasta(fasta_file, tsv_file, output_file)

    triplet_counter = Counter(generate_triplets())
    # Esegui il conteggio delle triplette nel file FASTA
    for record in SeqIO.parse(output_file, "fasta"):
      sequence = str(record.seq).upper()  # Converte la sequenza in maiuscolo
      for i in range(0, len(sequence) - 2, 3):
        triplet = sequence[i:i + 3]
        if triplet in triplet_counter:
          triplet_counter[triplet] += 1

    for key in triplet_counter:
       triplet_counter[key] -= 1

    def compare_blocks(block_BRCA, block_ESONI):
      merged_block = block_BRCA.merge(block_ESONI, on='chrom', how='inner')
      filtered_block = merged_block[
           (merged_block['pos_start'] >= merged_block['exonStarts']) &
           (merged_block['pos_end'] <= merged_block['exonEnds'])]
      return filtered_block[['Project', 'Sample', 'ID', 'Genome', 'mut_type', 'chrom', 'pos_start','pos_end', 'ref', 'alt', 'Type']]

    block_size = 1000

    num_blocks_BRCA = int(np.ceil(len(BRCA_FR_SUPERFILTRATO_WG) / block_size))
    num_blocks_ESONI = int(np.ceil(len(ESONI) / block_size))

    block_indices = []
    for i in range(num_blocks_BRCA):
      start_idx_BRCA = i * block_size
      end_idx_BRCA = min((i + 1) * block_size, len(BRCA_FR_SUPERFILTRATO_WG))

      for j in range(num_blocks_ESONI):
        start_idx_ESONI = j * block_size
        end_idx_ESONI = min((j + 1) * block_size, len(ESONI))

        block_indices.append([start_idx_BRCA, end_idx_BRCA, start_idx_ESONI, end_idx_ESONI])

    def compare_blocks_wrapper(indices):
        block_BRCA = BRCA_FR_SUPERFILTRATO_WG.iloc[indices[0]:indices[1]]
        block_ESONI = ESONI.iloc[indices[2]:indices[3]]
        return compare_blocks(block_BRCA, block_ESONI)

    with Pool() as pool:
      results = pool.map(compare_blocks_wrapper, block_indices)

    final_result = pd.concat(results, ignore_index=True)
    for i in range(final_result.shape[0]):
      final_result.loc[i,'chrom']= final_result.loc[i,'chrom'][3:]
    final_result.to_csv('/content/input/brca_fr_target_superfiltrato.txt', sep='\t', index=False)
    try:
      matrices = matGen.SigProfilerMatrixGeneratorFunc("test", "GRCh37", "/content",plot=False, exome=False, bed_file=None, chrom_based=False, tsb_stat=False, seqInfo=False, cushion=100)
    except ValueError as e:
      pass
    df = pd.read_table("/content/output/SBS/test.SBS96.all")
    df = df.T
    df.columns=df.iloc[0,:]
    df=df.iloc[1:73,]
    DF=DF.append(df)
    TRIPLETS[z]=triplet_counter

Elaborazione: 24390it [00:47, 516.58it/s] 


Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 2.73 seconds.
Matrices generated for 72 samples with 4946 errors. Total of 1368 SNVs, 6 DINUCs, and 0 INDELs were successfully analyzed.


Elaborazione: 28055it [00:20, 1384.99it/s]


Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 2.11 seconds.
Matrices generated for 71 samples with 5843 errors. Total of 1548 SNVs, 2 DINUCs, and 0 INDELs were successfully analyzed.


Elaborazione: 22224it [00:08, 2486.97it/s]


...

Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 2.0 seconds.
Matrices generated for 70 samples with 2457 errors. Total of 667 SNVs, 1 DINUCs, and 0 INDELs were successfully analyzed.


Elaborazione: 15709it [00:02, 6135.92it/s]


Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 2.01 seconds.
Matrices generated for 71 samples with 3144 errors. Total of 872 SNVs, 3 DINUCs, and 0 INDELs were successfully analyzed.


Elaborazione: 26976it [00:03, 7081.21it/s]


Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 2.09 seconds.
Matrices generated for 72 samples with 6017 errors. Total of 1586 SNVs, 6 DINUCs, and 0 INDELs were successfully analyzed.


In [57]:
DF.to_csv('DF.csv')

In [58]:
DF

,Unnamed: 0,A[C>A]A,A[C>A]C,A[C>A]G,A[C>A]T,A[C>G]A,A[C>G]C,A[C>G]G,A[C>G]T,A[C>T]A,...,T[T>A]G,T[T>A]T,T[T>C]A,T[T>C]C,T[T>C]G,T[T>C]T,T[T>G]A,T[T>G]C,T[T>G]G,T[T>G]T
0,SA594315,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SA594666,0,0,0,0,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
2,SA594675,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,SA594687,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SA594695,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70991,SA596039,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
70992,SA596051,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
70993,SA596068,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70994,SA596093,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [33]:
# Convertire il dizionario in una stringa JSON
import json
json_string = json.dumps(TRIPLETS, indent=2)  # indent è opzionale e rende il file più leggibile

# Salvare la stringa JSON in un file di testo
with open('TRIPLETS.txt', 'w') as file:
    file.write(json_string)

In [25]:
len(TRIPLETS)

600

In [29]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
import random

for z in range(800,1000):
    ESONI=pd.read_csv('/content/drive/MyDrive/esoni_noduplicati.tsv',sep='\t')
    ESONI['chrom']=ESONI['chrom'].astype(str)
    ESONI['exonStarts']=ESONI['exonStarts'].astype(int)
    ESONI['exonEnds']=ESONI['exonEnds'].astype(int)
    n=random.randint(7510,30038)
    ESONI=ESONI.sample(n)
    ESONI.to_csv("esoni_target.tsv",sep='\t', index=False)
    #triplette
    fasta_file = '/content/drive/MyDrive/hg19.fa'
    tsv_file = '/content/esoni_target.tsv'
    output_file = '/content/drive/MyDrive/esoni_genoma_triplette_targettato_ANN.fa'

    extract_sequences_from_fasta(fasta_file, tsv_file, output_file)

    triplet_counter = Counter(generate_triplets())
    # Esegui il conteggio delle triplette nel file FASTA
    for record in SeqIO.parse(output_file, "fasta"):
      sequence = str(record.seq).upper()  # Converte la sequenza in maiuscolo
      for i in range(0, len(sequence) - 2, 3):
        triplet = sequence[i:i + 3]
        if triplet in triplet_counter:
          triplet_counter[triplet] += 1

    for key in triplet_counter:
       triplet_counter[key] -= 1

    def compare_blocks(block_BRCA, block_ESONI):
      merged_block = block_BRCA.merge(block_ESONI, on='chrom', how='inner')
      filtered_block = merged_block[
           (merged_block['pos_start'] >= merged_block['exonStarts']) &
           (merged_block['pos_end'] <= merged_block['exonEnds'])]
      return filtered_block[['Project', 'Sample', 'ID', 'Genome', 'mut_type', 'chrom', 'pos_start','pos_end', 'ref', 'alt', 'Type']]

    block_size = 1000

    num_blocks_BRCA = int(np.ceil(len(BRCA_FR_SUPERFILTRATO_WG) / block_size))
    num_blocks_ESONI = int(np.ceil(len(ESONI) / block_size))

    block_indices = []
    for i in range(num_blocks_BRCA):
      start_idx_BRCA = i * block_size
      end_idx_BRCA = min((i + 1) * block_size, len(BRCA_FR_SUPERFILTRATO_WG))

      for j in range(num_blocks_ESONI):
        start_idx_ESONI = j * block_size
        end_idx_ESONI = min((j + 1) * block_size, len(ESONI))

        block_indices.append([start_idx_BRCA, end_idx_BRCA, start_idx_ESONI, end_idx_ESONI])

    def compare_blocks_wrapper(indices):
        block_BRCA = BRCA_FR_SUPERFILTRATO_WG.iloc[indices[0]:indices[1]]
        block_ESONI = ESONI.iloc[indices[2]:indices[3]]
        return compare_blocks(block_BRCA, block_ESONI)

    with Pool() as pool:
      results = pool.map(compare_blocks_wrapper, block_indices)

    final_result = pd.concat(results, ignore_index=True)
    for i in range(final_result.shape[0]):
      final_result.loc[i,'chrom']= final_result.loc[i,'chrom'][3:]
    final_result.to_csv('/content/input/brca_fr_target_superfiltrato.txt', sep='\t', index=False)
    try:
      matrices = matGen.SigProfilerMatrixGeneratorFunc("test", "GRCh37", "/content",plot=False, exome=False, bed_file=None, chrom_based=False, tsb_stat=False, seqInfo=False, cushion=100)
    except ValueError as e:
      pass
    df = pd.read_table("/content/output/SBS/test.SBS96.all")
    df = df.T
    df.columns=df.iloc[0,:]
    df=df.iloc[1:73,]
    DF=DF.append(df)
    TRIPLETS[z]=triplet_counter

Elaborazione: 10084it [00:47, 214.54it/s] 


Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 5.89 seconds.
Matrices generated for 70 samples with 2206 errors. Total of 576 SNVs, 0 DINUCs, and 0 INDELs were successfully analyzed.


Elaborazione: 16362it [00:20, 811.70it/s] 


Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 3.81 seconds.
Matrices generated for 72 samples with 3409 errors. Total of 964 SNVs, 3 DINUCs, and 0 INDELs were successfully analyzed.


Elaborazione: 20838it [00:13, 1558.48it/s]


...

Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 3.29 seconds.
Matrices generated for 70 samples with 3009 errors. Total of 703 SNVs, 1 DINUCs, and 0 INDELs were successfully analyzed.


Elaborazione: 29872it [00:07, 3913.80it/s]


Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 4.45 seconds.
Matrices generated for 72 samples with 6293 errors. Total of 1579 SNVs, 1 DINUCs, and 0 INDELs were successfully analyzed.


Elaborazione: 21896it [00:08, 2544.45it/s]


Starting matrix generation for SNVs and DINUCs...Completed! Elapsed time: 3.87 seconds.
Matrices generated for 72 samples with 4647 errors. Total of 1263 SNVs, 2 DINUCs, and 0 INDELs were successfully analyzed.


In [30]:
len(TRIPLETS)

1000

In [31]:
DF

,Unnamed: 0,A[C>A]A,A[C>A]C,A[C>A]G,A[C>A]T,A[C>G]A,A[C>G]C,A[C>G]G,A[C>G]T,A[C>T]A,...,T[T>A]G,T[T>A]T,T[T>C]A,T[T>C]C,T[T>C]G,T[T>C]T,T[T>G]A,T[T>G]C,T[T>G]G,T[T>G]T
0,SA594315,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,SA594666,0,0,0,0,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
2,SA594675,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,SA594687,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SA594695,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SA596039,NaN,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
SA596051,NaN,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
SA596068,NaN,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SA596093,NaN,0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
